In [36]:
import time      # for testing use only
import os         # for testing use only

from bs4 import BeautifulSoup as bs
import pandas as pd
import scipy as sc
import numpy as np
import requests
from lxml import html
import csv
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [37]:
def load_csv(file_name):
    df = pd.read_csv(file_name)
    return df

In [38]:
df = load_csv('FixData.csv')
df

,Name,Location,Vegetarian,Gluten,FoodRating,ServiceFood,ValueFood,RankingInArea,Ratings,CuisineCount,NumOfReviews,price,Origin,Type
0,Burger & Lobster,Midtown West,0,0,4.5,4.5,4.5,129,4.5,2,"1,078",2.5,American,Seafood
1,Club A Steakhouse,Flatiron District,0,1,4.5,4.5,4.5,1,4.5,2,"4,158",4.0,American,Steakhouse
2,Au Za'atar,Midtown East,0,0,4.5,4.0,4.0,2,4.5,2,247,2.5,Lebanese,Mediterranean
3,Boucherie Union Square,Downtown Manhattan (Downtown),0,1,4.5,4.5,4.0,3,4.5,2,"1,065",4.0,French,Steakhouse
4,Mei Jin Ramen,Flatiron District,0,0,4.5,4.5,4.5,4,4.5,2,173,2.5,Japanese,Asian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5909,Quality Meats,Greenwich Village,0,1,4.5,4.5,4.0,471,4.5,2,"2,377",4.0,American,Steakhouse
5910,Il Punto,East Harlem,0,0,4.5,4.5,4.0,45,4.5,2,"1,517",2.5,Italian,Mediterranean
5911,Smith & Wollensky,Morningside Heights,0,1,4.0,4.0,3.5,838,4.0,2,"3,285",4.0,American,Steakhouse
5912,Chocolat Restaurant & Bar,Upper West Side,0,0,4.0,3.5,3.5,"4,053",3.5,2,123,2.5,American,Barbecue


In [142]:
df.columns

Index(['Name', 'Location', 'Vegetarian', 'Gluten', 'FoodRating', 'ServiceFood',
       'ValueFood', 'RankingInArea', 'Ratings', 'CuisineCount', 'NumOfReviews',
       'price', 'Origin', 'Type'],
      dtype='object')

In [132]:
df.iloc[:,4:9].values

array([[4.5, 4.5, 4.5, '129', 4.5],
       [4.5, 4.5, 4.5, '1', 4.5],
       [4.5, 4.0, 4.0, '2', 4.5],
       ...,
       [0.0, 0.0, 0.0, '0', 0.0],
       [0.0, 0.0, 0.0, '0', 0.0],
       [5.0, 5.0, 5.0, '2,894', 4.5]], dtype=object)

In [147]:
dfcolumns= ['FoodRating', 'ServiceFood',
       'ValueFood', 'RankingInArea', 'Ratings', 'CuisineCount', 'NumOfReviews',
       'price']
for col in dfcolumns:
    df[col] = df[col].replace({0: None})

#df['FoodRating'] = df['FoodRating'].replace({0: None})

In [148]:
df

,Name,Location,Vegetarian,Gluten,FoodRating,ServiceFood,ValueFood,RankingInArea,Ratings,CuisineCount,NumOfReviews,price,Origin,Type
0,Burger & Lobster,Midtown West,0,0,4.5,4.5,4.5,129,4.5,2,"1,078",2.5,American,Seafood
1,Club A Steakhouse,Flatiron District,0,1,4.5,4.5,4.5,1,4.5,2,"4,158",4.0,American,Steakhouse
2,Au Za'atar,Midtown East,0,0,4.5,4.0,4.0,2,4.5,2,247,2.5,Lebanese,Mediterranean
3,Boucherie Union Square,Downtown Manhattan (Downtown),0,1,4.5,4.5,4.0,3,4.5,2,"1,065",4.0,French,Steakhouse
4,Mei Jin Ramen,Flatiron District,0,0,4.5,4.5,4.5,4,4.5,2,173,2.5,Japanese,Asian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12385,Blimpie,Downtown Manhattan (Downtown),0,0,None,None,None,0,None,1,,None,,NaN
12386,Number Chinese Restaurant,Upper Manhattan (Uptown),0,0,None,None,None,0,None,1,,NaN,Chinese,NaN
12387,Great Sichuan,Harlem,0,0,None,None,None,0,None,1,,NaN,,NaN
12388,Sedapsss Fusion Cuisine,Kips Bay,0,0,None,None,None,0,None,2,,NaN,Chinese,Japanese


In [161]:
df.isnull().sum()

Name                4
Location          254
Vegetarian          0
Gluten              0
FoodRating       5437
ServiceFood      5459
ValueFood        5538
RankingInArea       0
Ratings          1145
CuisineCount        0
NumOfReviews        0
price            3739
Origin           2229
Type             5528
dtype: int64

In [159]:
dfcolumns= ['Name', 'Location', 'Origin', 'Type']
for col in dfcolumns:
    df[col] = df[col].replace({" ": None}).replace({"": None}).replace({" ": None})

In [160]:
df.duplicated().sum()

1203

In [207]:
df

,Name,Location,Vegetarian,Gluten,FoodRating,ServiceFood,ValueFood,RankingInArea,Ratings,CuisineCount,NumOfReviews,price,Origin,Type
0,Burger & Lobster,Midtown West,0,0,4.5,4.5,4.5,129,4.5,2,"1,078",2.5,American,Seafood
1,Club A Steakhouse,Flatiron District,0,1,4.5,4.5,4.5,1,4.5,2,"4,158",4.0,American,Steakhouse
2,Au Za'atar,Midtown East,0,0,4.5,4.0,4.0,2,4.5,2,247,2.5,Lebanese,Mediterranean
3,Boucherie Union Square,Downtown Manhattan (Downtown),0,1,4.5,4.5,4.0,3,4.5,2,"1,065",4.0,French,Steakhouse
4,Mei Jin Ramen,Flatiron District,0,0,4.5,4.5,4.5,4,4.5,2,173,2.5,Japanese,Asian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5909,Quality Meats,Greenwich Village,0,1,4.5,4.5,4.0,471,4.5,2,"2,377",4.0,American,Steakhouse
5910,Il Punto,East Harlem,0,0,4.5,4.5,4.0,45,4.5,2,"1,517",2.5,Italian,Mediterranean
5911,Smith & Wollensky,Morningside Heights,0,1,4.0,4.0,3.5,838,4.0,2,"3,285",4.0,American,Steakhouse
5912,Chocolat Restaurant & Bar,Upper West Side,0,0,4.0,3.5,3.5,"4,053",3.5,2,123,2.5,American,Barbecue


In [175]:
df = df.dropna(axis=0, thresh=13)

In [177]:
df

,Name,Location,Vegetarian,Gluten,FoodRating,ServiceFood,ValueFood,RankingInArea,Ratings,CuisineCount,NumOfReviews,price,Origin,Type
0,Burger & Lobster,Midtown West,0,0,4.5,4.5,4.5,129,4.5,2,"1,078",2.5,American,Seafood
1,Club A Steakhouse,Flatiron District,0,1,4.5,4.5,4.5,1,4.5,2,"4,158",4.0,American,Steakhouse
2,Au Za'atar,Midtown East,0,0,4.5,4.0,4.0,2,4.5,2,247,2.5,Lebanese,Mediterranean
3,Boucherie Union Square,Downtown Manhattan (Downtown),0,1,4.5,4.5,4.0,3,4.5,2,"1,065",4.0,French,Steakhouse
4,Mei Jin Ramen,Flatiron District,0,0,4.5,4.5,4.5,4,4.5,2,173,2.5,Japanese,Asian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12359,Quality Meats,Upper Manhattan (Uptown),0,1,4.5,4.5,4.0,471,4.5,2,"2,377",4.0,American,Steakhouse
12365,Smith & Wollensky,Morningside Heights,0,1,4.0,4.0,3.5,838,4.0,2,"3,285",4.0,American,Steakhouse
12371,Quality Bistro,Upper East Side,0,0,4.5,5.0,4.0,"2,104",4.5,2,27,2.5,French,Steakhouse
12383,Chocolat Restaurant & Bar,Upper West Side,0,0,4.0,3.5,3.5,"4,053",3.5,2,123,2.5,American,Barbecue


In [208]:
df.duplicated().sum()

0

In [180]:
df = df.drop_duplicates()

In [182]:
df.duplicated().sum()

0

In [183]:
df.describe(include='all')

,Name,Location,Vegetarian,Gluten,FoodRating,ServiceFood,ValueFood,RankingInArea,Ratings,CuisineCount,NumOfReviews,price,Origin,Type
count,5913,5883,5914.000000,5914.000000,5914.0,5914.0,5883.0,5914,5914.0,5914.000000,5914,5882.0,5914,4423
unique,4659,86,NaN,NaN,7.0,7.0,8.0,2674,7.0,NaN,859,3.0,87,92
top,Burger & Lobster,Downtown Manhattan (Downtown),NaN,NaN,4.5,4.5,4.0,7,4.0,NaN,24,2.5,American,Bar
freq,49,1116,NaN,NaN,2835.0,2395.0,2961.0,47,2713.0,NaN,100,4030.0,1609,582
mean,NaN,NaN,0.322962,0.059182,NaN,NaN,NaN,NaN,NaN,1.747886,NaN,NaN,NaN,NaN
std,NaN,NaN,0.467648,0.235984,NaN,NaN,NaN,NaN,NaN,0.434263,NaN,NaN,NaN,NaN
min,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN


In [184]:
fileName='data.csv'
df.to_csv(fileName, encoding='utf-8', index=False)

In [209]:
for col in df.columns:
        if np.issubdtype(df[col].dtype, np.floating) or np.issubdtype(df[col].dtype, np.integer):
            Q1 = np.percentile(df[col], 25)
            Q3 = np.percentile(df[col], 75)
            IQR = Q3 - Q1
            IQR_range = 1.5 * IQR
            df[col][(df[col] < Q1 - IQR_range) | (df[col] > Q3 + IQR_range )] = np.nan
    

<ipython-input-209-c1b003fc9b18>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][(df[col] < Q1 - IQR_range) | (df[col] > Q3 + IQR_range )] = np.nan
<ipython-input-209-c1b003fc9b18>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][(df[col] < Q1 - IQR_range) | (df[col] > Q3 + IQR_range )] = np.nan
<ipython-input-209-c1b003fc9b18>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][(df[col] < Q1 - IQR_range) | (df[co

In [212]:
df

,Name,Location,Vegetarian,Gluten,FoodRating,ServiceFood,ValueFood,RankingInArea,Ratings,CuisineCount,NumOfReviews,price,Origin,Type
0,Burger & Lobster,Midtown West,0,0,4.5,4.5,4.5,129,4.5,2,"1,078",2.5,American,Seafood
1,Club A Steakhouse,Flatiron District,0,1,4.5,4.5,4.5,1,4.5,2,"4,158",4.0,American,Steakhouse
2,Au Za'atar,Midtown East,0,0,4.5,4.0,4.0,2,4.5,2,247,2.5,Lebanese,Mediterranean
3,Boucherie Union Square,Downtown Manhattan (Downtown),0,1,4.5,4.5,4.0,3,4.5,2,"1,065",4.0,French,Steakhouse
4,Mei Jin Ramen,Flatiron District,0,0,4.5,4.5,4.5,4,4.5,2,173,2.5,Japanese,Asian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5909,Quality Meats,Greenwich Village,0,1,4.5,4.5,4.0,471,4.5,2,"2,377",4.0,American,Steakhouse
5910,Il Punto,East Harlem,0,0,4.5,4.5,4.0,45,4.5,2,"1,517",2.5,Italian,Mediterranean
5911,Smith & Wollensky,Morningside Heights,0,1,4.0,4.0,3.5,838,4.0,2,"3,285",4.0,American,Steakhouse
5912,Chocolat Restaurant & Bar,Upper West Side,0,0,4.0,3.5,3.5,"4,053",3.5,2,123,2.5,American,Barbecue


In [199]:
import seaborn as sns

In [215]:
df['Type'].value_counts()

 Bar              582
 Asian            409
 Pizza            328
 Mediterranean    317
 Steakhouse       243
                 ... 
 Uzbek              1
 Central Asian      1
 Czech              1
 Colombian          1
 Hawaiian           1
Name: Type, Length: 92, dtype: int64

In [216]:
df['Origin'].value_counts()

American        1609
Italian          987
Japanese         378
Bar              265
Chinese          254
                ... 
Swedish            1
Scottish           1
Polish             1
Puerto Rican       1
Canadian           1
Name: Origin, Length: 87, dtype: int64

In [217]:
df['Location'].value_counts()

Downtown Manhattan (Downtown)    1116
Midtown                           614
Midtown West                      436
Midtown East                      398
Upper East Side                   363
                                 ... 
Flatbush                            1
Bensonhurst                         1
Governors Island                    1
Mount Hope                          1
Sheepshead Bay                      1
Name: Location, Length: 86, dtype: int64

In [5]:
df

,Name,Location,Vegetarian,Gluten,FoodRating,ServiceFood,ValueFood,RankingInArea,Ratings,CuisineCount,NumOfReviews,price,Origin,Type
0,Burger & Lobster,Midtown West,0,0,4.5,4.5,4.5,129,4.5,2,"1,078",2.5,American,Seafood
1,Club A Steakhouse,Flatiron District,0,1,4.5,4.5,4.5,1,4.5,2,"4,158",4.0,American,Steakhouse
2,Au Za'atar,Midtown East,0,0,4.5,4.0,4.0,2,4.5,2,247,2.5,Lebanese,Mediterranean
3,Boucherie Union Square,Downtown Manhattan (Downtown),0,1,4.5,4.5,4.0,3,4.5,2,"1,065",4.0,French,Steakhouse
4,Mei Jin Ramen,Flatiron District,0,0,4.5,4.5,4.5,4,4.5,2,173,2.5,Japanese,Asian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5909,Quality Meats,Greenwich Village,0,1,4.5,4.5,4.0,471,4.5,2,"2,377",4.0,American,Steakhouse
5910,Il Punto,East Harlem,0,0,4.5,4.5,4.0,45,4.5,2,"1,517",2.5,Italian,Mediterranean
5911,Smith & Wollensky,Morningside Heights,0,1,4.0,4.0,3.5,838,4.0,2,"3,285",4.0,American,Steakhouse
5912,Chocolat Restaurant & Bar,Upper West Side,0,0,4.0,3.5,3.5,"4,053",3.5,2,123,2.5,American,Barbecue


In [97]:
dfl = df["Location"].tolist()

for i,n in enumerate(dfl):
    try:
        dfl[i] = dfl[i].replace(" (Downtown)","")
    except:
        dfl[i] = None
dfl

['Midtown West',
 'Flatiron District',
 'Midtown East',
 'Downtown Manhattan',
 'Flatiron District',
 'Upper East Side',
 'Downtown Manhattan',
 'Chelsea',
 'West Village',
 'Downtown Manhattan',
 'West Village',
 'Midtown East',
 'Midtown',
 'Downtown Manhattan',
 'West Village',
 'Harlem',
 'Upper East Side',
 'Hudson Yards',
 'Midtown West',
 'Murray Hill',
 'Downtown Manhattan',
 'Downtown Manhattan',
 'Murray Hill',
 'Midtown East',
 'Hudson Yards',
 'Midtown West',
 'Murray Hill',
 'Kips Bay',
 'West Village',
 'Downtown Manhattan',
 'Greenwich Village',
 'Downtown Manhattan',
 'Midtown',
 'Upper East Side',
 'Murray Hill',
 'West Village',
 'Midtown',
 'Downtown Manhattan',
 'Midtown',
 'Midtown West',
 'Midtown',
 'Midtown',
 'Upper East Side',
 'Downtown Manhattan',
 'Greenwich Village',
 'West Village',
 'Midtown West',
 'Midtown',
 'Downtown Manhattan',
 'Flatiron District',
 'Midtown',
 'Downtown Manhattan',
 'Midtown West',
 'Nolita',
 'Hudson Yards',
 'Theater District',


In [98]:
from geopy.geocoders import Nominatim
latitude=[]
longitude=[]
for city in dfl:
    address = city + ', New York'
    geolocator = Nominatim(user_agent="tor_explorer")
    location = geolocator.geocode(address)
    if location:
        print(address)
        latitude.append(location.latitude)
        longitude.append(location.longitude)


Midtown West, New York
Flatiron District, New York
Midtown East, New York
Downtown Manhattan, New York
Flatiron District, New York
Upper East Side, New York
Downtown Manhattan, New York
Chelsea, New York
West Village, New York
Downtown Manhattan, New York
West Village, New York
Midtown East, New York
Midtown, New York
Downtown Manhattan, New York
West Village, New York
Harlem, New York
Upper East Side, New York
Hudson Yards, New York
Midtown West, New York
Murray Hill, New York
Downtown Manhattan, New York
Downtown Manhattan, New York
Murray Hill, New York
Midtown East, New York
Hudson Yards, New York
Midtown West, New York
Murray Hill, New York
Kips Bay, New York
West Village, New York
Downtown Manhattan, New York
Greenwich Village, New York
Downtown Manhattan, New York
Midtown, New York
Upper East Side, New York
Murray Hill, New York
West Village, New York
Midtown, New York
Downtown Manhattan, New York
Midtown, New York
Midtown West, New York
Midtown, New York
Midtown, New York
Upper

Murray Hill, New York
Murray Hill, New York
Midtown West, New York
Upper East Side, New York
Midtown, New York
Flatiron District, New York
Midtown East, New York
Hudson Square, New York
Midtown, New York
Kips Bay, New York
Midtown West, New York
Midtown West, New York
Midtown East, New York
Midtown West, New York
Downtown Manhattan, New York
Downtown Manhattan, New York
Hudson Yards, New York
NoMad, New York
Midtown, New York
Theater District, New York
Midtown, New York
Theater District, New York
Theater District, New York
Greenwich Village, New York
Midtown East, New York
Midtown West, New York
Downtown Manhattan, New York
Nolita, New York
Downtown Manhattan, New York
Downtown Manhattan, New York
Theater District, New York
Kips Bay, New York
Theater District, New York
Downtown Manhattan, New York
Midtown West, New York
Murray Hill, New York
Downtown Manhattan, New York
Flatiron District, New York
Midtown, New York
Williamsburg, New York
Midtown West, New York
Flatiron District, New Yo

Flatiron District, New York
Midtown East, New York
Upper West Side, New York
Downtown Manhattan, New York
Midtown East, New York
Downtown Manhattan, New York
Midtown West, New York
Upper East Side, New York
Murray Hill, New York
Midtown East, New York
Downtown Manhattan, New York
Flatiron District, New York
Midtown West, New York
Hudson Yards, New York
Midtown, New York
Theater District, New York
Theater District, New York
Downtown Manhattan, New York
West Village, New York
Central Park, New York
Chinatown, New York
Downtown Manhattan, New York


TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [85]:
longitude

[-73.9923918, -73.9896534, -73.9724708, -74.0139589, -73.9896534]

In [41]:
longitude

[-73.9923918, -73.9896534, -73.9724708, -74.00807244760026, -73.9896534]

In [86]:
latitude

[40.7644228, 40.7410722, 40.7598219, 40.7130418, 40.7410722]

In [42]:
latitude

[40.7644228, 40.7410722, 40.7598219, 40.7012724, 40.7410722]

In [87]:
lo = geolocator.geocode('Downtown Manhattan, New York')

In [88]:
lo

Location(Downtown Hudson Tubes, West Street, Manhattan Community Board 1, Manhattan, New York, 10281, United States, (40.7130418, -74.0139589, 0.0))